# 第五章 搭建一个多文本智能体 Building a Multi-Document Agent

 - [一、引言](#一、引言)
 - [二、环境配置](#三、定义模型)
 - [三、三篇文章的智能体实现](#二、加载数据)
     - [3.1 数据加载](#2.1-加载数据为文档格式)
     - [3.2 工具配置](#2.2-切割文档为节点格式)
     - [3.3 智能体搭建](#2.3-定义模型)
     - [3.4 查询实验](#2.4-训练模型)
 - [四、无限文章的智能体实现](#三、定义模型)
     - [4.1 数据加载](#2.1-加载数据为文档格式)
     - [4.2 工具配置](#2.2-切割文档为节点格式)
     - [4.3 扩展智能体与工具检索](#2.3-定义模型)
     - [4.3 智能体搭建](#2.3-定义模型)
     - [4.4 查询实验](#2.4-训练模型)
 - [五、总结](#七、结合起来)

## 一、引言
在本章节中，我们将学习一个多篇文章组成的智能体。从3篇文章入手学习简单多文章智能体的搭建，通过向量工具和总结工具配合组成工具列表，通过query对文章提问得到结果。接着从11篇文章入手学习复杂多文章智能体的搭建，通过配置对象索引和retrieve检索的方式完成对多个文章提问得到结果。

原视频来自：https://learn.deeplearning.ai/courses/building-agentic-rag-with-llamaindex/lesson/5/building-a-multi-document-agent


## 二、环境配置

In [1]:
# 载入模型 API 相关环境变量
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [4]:
# 配置嵌套异步

import nest_asyncio
nest_asyncio.apply()

In [2]:
# 安装相关依赖
!pip install python-dotenv==1.0.0  llama-index==0.10.27  llama-index-llms-openai==0.1.15  llama-index-embeddings-openai==0.1.7 alpaca-py==0.14.0 requests --user

^C


## 三、三篇文章的智能体实现
### 3.1 数据加载 

**Note**: The pdf files are included with this lesson. To access these papers, go to the `File` menu and select`Open...`.

In [25]:
urls = [
    "https://openreview.net/pdf?id=VtmBAGCN7o",
    "https://openreview.net/pdf?id=6PmJoRfdaK",
    "https://openreview.net/pdf?id=hSyW5go0v8",
]

papers = [
    "metagpt.pdf",
    "longlora.pdf",
    "selfrag.pdf",
]

In [17]:
# 文章下载脚本，如果有可以跳过此步骤。可能需要解决网络问题。
import requests


for url, paper in zip(urls, papers):
    response = requests.get(url)
    with open(paper, 'wb') as file:
        file.write(response.content)

KeyboardInterrupt: 

### 3.2 工具配置

In [23]:
# 接下来，我们将把每篇论文转化为一个工具。

from utils import get_doc_tools
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    # get_doc_tools函数，它会自动构建向量索引工具以及给定论文的摘要索引工具。 因此向量工具执行向量搜索。汇总工具对整个文档进行汇总。
    #于每篇论文，我们都会返回向量工具和摘要工具。
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: metagpt.pdf
Getting tools for paper: longlora.pdf
Getting tools for paper: loftq.pdf
Getting tools for paper: swebench.pdf
Getting tools for paper: selfrag.pdf
Getting tools for paper: zipformer.pdf
Getting tools for paper: values.pdf
Getting tools for paper: finetune_fair_diffusion.pdf
Getting tools for paper: knowledge_card.pdf
Getting tools for paper: metra.pdf
Getting tools for paper: vr_mcl.pdf


In [7]:
# 我们把它放入这个整体词典中，映射每篇论文向量工具和汇总工具
initial_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

In [8]:
# OpenAI 的 3.5 Turbo 作为我们的大语言模型支持
from llama_index.llms.openai import OpenAI

# llm = OpenAI(model="gpt-3.5-turbo",api_base='https://api.gpts.vin/v1')
llm = OpenAI(model="gpt-3.5-turbo")

In [9]:
# 三组向量工具和汇总工具共有6个工具。

len(initial_tools)

6

### 3.3 智能体搭建

In [10]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

# 下一步构建整体任务的智能体agent_worker
agent_worker = FunctionCallingAgentWorker.from_tools(
    initial_tools, 
    llm=llm, 
    verbose=True
)
# agent_worker智能体包括六个工具以及刚配置的LLM
agent = AgentRunner(agent_worker)

### 3.4 查询实验

下面是一个测试向量工具的问题，针对LongLoRA中使用的评估数据集简单测试。

In [11]:
response = agent.query(
    "告诉我LongLoRA中使用的评估数据集, "
    "然后告诉我评估结果"
)

Added user message to memory: 告诉我LongLoRA中使用的评估数据集, 然后告诉我评估结果
=== Calling Function ===
Calling function: vector_tool_longlora with args: {"query": "evaluation dataset"}
=== Function Output ===
The proposed dataset in the context is called SAFECONV.
=== Calling Function ===
Calling function: vector_tool_longlora with args: {"query": "evaluation results"}
=== Function Output ===
The evaluation results show that the proposed model, labeled as "Ours-7B," performs comparably or even better than other Llama2-based long-context models like Vicuna and LongChat on long-context benchmarks such as LongBench and LEval. The model demonstrates competitive performance while being efficient, requiring only about 4 hours and approximately 0.3 billion tokens on a single 8 × A100 machine.
=== LLM Response ===
LongLoRA中使用的评估数据集是SAFECONV，评估结果显示，提出的模型在长上下文基准测试中表现出色，标记为“Ours-7B”的模型在长上下文基准测试如LongBench和LEval上表现出色，与其他基于Llama2的长上下文模型如Vicuna和LongChat相比表现相当甚至更好。该模型表现出色且高效，仅需约4小时和大约0.3亿个标记在单个8×A100机器上。


下面是针对总结工具的测试，对Self-RAG 与 LongLoRA提出总结需求

In [12]:
response = agent.query("给我Self-RAG 与 LongLoRA的总结")
print(str(response))

Added user message to memory: 给我Self-RAG 与 LongLoRA的总结
=== Calling Function ===
Calling function: summary_tool_selfrag with args: {"input": "Summary"}
=== Function Output ===
The SELF-RAG framework is a model designed for text generation tasks that aims to enhance the quality and factuality of Language Models (LMs) through retrieval on demand and self-reflection. It consists of a generator model, a retriever model, and a critic model, trained using a curated corpus with interleaving passages retrieved by the retriever and reflection tokens predicted by the critic. The system enables the tailoring of LM behaviors at test time by leveraging reflection tokens, allowing it to adapt to different task requirements. Experimental results show that SELF-RAG significantly outperforms state-of-the-art LLMs and retrieval-augmented models on various tasks, demonstrating superior performance in tasks like PubHealth, PopQA, biography generations, and ASQA.
=== Calling Function ===
Calling function: s

译文

新增用户消息到内存:给我Self-RAG与LongLoRA的总结
===调用函数===
调用函数:summary_tool_selfrag，参数为:{"input": "Summary"}
===函数输出===
SELF-RAG框架是一个为文本生成任务设计的模型，旨在通过按需检索和自我反思来提高语言模型(LMs)的质量和真实性。它由一个生成器模型、一个检索器模型和一个评论家模型组成，使用一个经过整理的语料库进行训练，语料库中包含检索器检索的交错段落和评论家预测的反射标记。通过利用反射令牌，系统可以在测试时对LM行为进行裁剪，从而适应不同的任务需求。实验结果表明，SELF-RAG在各种任务上明显优于最先进的llm和检索增强模型，在PubHealth、PopQA、传记生成和ASQA等任务中表现出优异的性能。
===调用函数===
调用函数:summary_tool_longlora，参数为:{"input": "Summary"}
===函数输出===
LongLoRA和S2-Attn等高效的微调方法已经被开发出来，用于扩展预训练大型语言模型的上下文大小，同时降低计算成本。LongLoRA将改进的LoRA与S2-Attn相结合，在各种任务中获得了强有力的实证结果。该方法支持对大型语言模型(如Llama2)的上下文窗口进行有效扩展，在减少计算资源的情况下保持性能。该研究还涉及了长上下文语言建模任务的实验，证明了更长的上下文大小可以提高性能。此外，这些论文涵盖了深度学习和自然语言处理的一系列主题，展示了模型、技术和评估基准的进步。
=== LLM响应===
SELF-RAG框架是一个为文本生成任务设计的模型，旨在通过按需检索和自我反思来提高语言模型(LMs)的质量和真实性。它由一个生成器模型、一个检索器模型和一个评论家模型组成，使用一个经过整理的语料库进行训练，语料库中包含检索器检索的交错段落和评论家预测的反射标记。通过利用反射令牌，系统可以在测试时对LM行为进行裁剪，从而适应不同的任务需求。实验结果表明，SELF-RAG在各种任务上明显优于最先进的llm和检索增强模型，在PubHealth、PopQA、传记生成和ASQA等任务中表现出优异的性能。

LongLoRA和S2-Attn等高效的微调方法已经被开发出来，用于扩展预训练大型语言模型的上下文大小，同时降低计算成本。LongLoRA将改进的LoRA与S2-Attn相结合，在各种任务中获得了强有力的实证结果。该方法支持对大型语言模型(如Llama2)的上下文窗口进行有效扩展，在减少计算资源的情况下保持性能。该研究还涉及了长上下文语言建模任务的实验，证明了更长的上下文大小可以提高性能。此外，这些论文涵盖了深度学习和自然语言处理的一系列主题，展示了模型、技术和评估基准的进步。


**小结**

可以看到在向量查询和总结查询两种任务上都有不错的表现，在llamaIndex工具中的tools及FunctionCallingAgentWorker框架能快速搭建一个简单的多文章智能体工具。

## 四、 无限文章的智能体实现

这里以11篇文章为例，通过完成对11篇文章实现多文章智能体工具可以扩展到更多文章智能体的搭建。

### 4.1 数据加载

In [13]:
urls = [
    "https://openreview.net/pdf?id=VtmBAGCN7o",
    "https://openreview.net/pdf?id=6PmJoRfdaK",
    "https://openreview.net/pdf?id=LzPWWPAdY4",
    "https://openreview.net/pdf?id=VTF8yNQM66",
    "https://openreview.net/pdf?id=hSyW5go0v8",
    "https://openreview.net/pdf?id=9WD9KwssyT",
    "https://openreview.net/pdf?id=yV6fD7LYkF",
    "https://openreview.net/pdf?id=hnrB5YHoYu",
    "https://openreview.net/pdf?id=WbWtOYIzIK",
    "https://openreview.net/pdf?id=c5pwL0Soay",
    "https://openreview.net/pdf?id=TpD2aG1h0D"
]

papers = [
    "metagpt.pdf",
    "longlora.pdf",
    "loftq.pdf",
    "swebench.pdf",
    "selfrag.pdf",
    "zipformer.pdf",
    "values.pdf",
    "finetune_fair_diffusion.pdf",
    "knowledge_card.pdf",
    "metra.pdf",
    "vr_mcl.pdf"
]

In [14]:
#文章下载

import requests


for url, paper in zip(urls, papers):
    response = requests.get(url)
    with open(paper, 'wb') as file:
        file.write(response.content)


KeyboardInterrupt: 

### 4.2 工具配置

这里与之前相同，先加载向量工具和总结工具。

In [26]:
# 接下来，我们将把每篇论文转化为一个工具。

from utils import get_doc_tools
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    # get_doc_tools函数，它会自动构建向量索引工具以及给定论文的摘要索引工具。 因此向量工具执行向量搜索。汇总工具对整个文档进行汇总。
    #每篇论文，我们都会返回向量工具和摘要工具。甚至你可以尝试更多文章。
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: metagpt.pdf
Getting tools for paper: longlora.pdf
Getting tools for paper: selfrag.pdf


### 4.3扩展智能体与工具检索

In [18]:
# 将工具转化为22个子工具的工具列表  
all_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

特别是如果您的文件数量很大，您将每个文档建模为一个单独的工具或一组工具，但由于数量增加，成本和延迟将会激增。大纲实际上可能会变得混乱，LLM 可能会失败。
当选择数量过多时，选择正确的工具。 这里的一个解决方案是，当用户提出查询时，我们实际上执行 Retrieval增强，但不是在文字层面，而是实际上在工具层面上我们首先检索一小组相关工具。
然后提供智能体推理提示的相关工具而不是所有工具。 该检索过程与 RAG 中使用的检索过程类似。最简单的，就是top k向量搜索。

In [19]:
# 针对工具定义对象索引
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [20]:
# 使用as_retriever增强数据

obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [24]:
#使用retrieve检索
tools = obj_retriever.retrieve(
    "告诉我在MetaGPT 和 SWE-Bench使用的数据集是什么"
)

In [45]:
# 返回检索得到的最后一条信息

tools[2].metadata

ToolMetadata(description='Useful for summarization questions related to longlora', name='summary_tool_longlora', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)

### 4.4 智能体搭建

In [46]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

# 搭建多篇文章的索引智能体
agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm, 
    # 引入系统prompt
    system_prompt=""" \
你是一个被设计用来回答对一组给定文件的查询的智能体。
请始终使用提供的工具来回答问题。不要依赖于先验知识\

""",
    verbose=True
)
agent = AgentRunner(agent_worker)

### 4.4 查询实验

通过两个综合例子可以看到11篇这样多的文章智能体也能很好的完成查询任务。

In [21]:


response = agent.query(
    "告诉我在MetaGPT中使用的评估数据集"
    "并将其与SWE-Bench进行比较"
)
print(str(response))

Added user message to memory: 告诉我在MetaGPT中使用的评估数据集并将其与SWE-Bench进行比较
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "evaluation dataset used in MetaGPT"}
=== Function Output ===
The evaluation dataset used in MetaGPT is the SoftwareDev dataset, which consists of 70 diverse software development tasks.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "comparison of evaluation dataset between MetaGPT and SWE-Bench"}
=== Function Output ===
MetaGPT is evaluated on the HumanEval and MBPP datasets, while SWE-Bench is not specifically mentioned in the provided context.
=== LLM Response ===
在MetaGPT中使用的评估数据集是SoftwareDev数据集，其中包含70个不同的软件开发任务。MetaGPT在HumanEval和MBPP数据集上进行评估，而SWE-Bench在提供的上下文中没有特别提到。
在MetaGPT中使用的评估数据集是SoftwareDev数据集，其中包含70个不同的软件开发任务。MetaGPT在HumanEval和MBPP数据集上进行评估，而SWE-Bench在提供的上下文中没有特别提到。


In [22]:
response = agent.query(
    "比较和对比LoRA论文(LongLoRA, LoftQ)。"
    "首先分析每篇论文中的方法。"
)

Added user message to memory: 比较和对比LoRA论文(LongLoRA, LoftQ)。首先分析每篇论文中的方法。
=== Calling Function ===
Calling function: vector_tool_longlora with args: {"query": "methods"}
=== Function Output ===
The proposed methods in the paper include the Action Units Relation Transformer (ART) for modeling relations between different facial action units at AU-agnostic patches and AU-specific regions, and the Tampered AU Prediction (TAP) for tampering AU-related regions to provide local tampering supervision. These methods contribute to capturing intra-face relations useful for forgery detection and improving the generalization ability of the model to unseen manipulation methods.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "methods"}
=== Function Output ===
The methods involved in the software development process in MetaGPT include analyzing user requirements, formulating a detailed PRD, translating requirements into system design components, task distribution by

## 五、总结

本小结结合前面学习的知识，通过拓展完成了多篇文章的智能体检索工具搭建。当文章太多的时候，加载所有工具变得十分困难。这个时候需要我们使用对象索引，获取前面最相关的几个工具，然后进行检索。从而达到更好的效果。